In [4]:
# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service, retrieve_query

service = get_tap_service()
assert service is not None
assert service.baseurl == "https://rsp.lsst.ac.uk/api/tap"

query = "SELECT * FROM tap_schema.schemas"
resultsSchema = service.search(query).to_table()
resultsSchema

description,schema_index,schema_name,utype
str512,int32,str64,str512
Data Preview 0.1 includes five tables based on the DESC's Data Challenge 2 simulation of 300 square degrees of the wide-fast-deep LSST survey region after 5 years. All tables contain objects detected in coadded images.,1,dp01_dc2_catalogs,
"Data Preview 0.2 contains the image and catalog products of the Rubin Science Pipelines v23 processing of the DESC Data Challenge 2 simulation, which covered 300 square degrees of the wide-fast-deep LSST survey region over 5 years.",0,dp02_dc2_catalogs,
ObsCore v1.1 attributes in ObsTAP realization,3,ivoa,
WP3.5 SXDS region data,2,SXDS,
A TAP-standard-mandated schema to describe tablesets in a TAP 1.1 service,100000,tap_schema,
UWS Metadata,120000,uws,


In [2]:
query = "SELECT TOP 10 * from SXDS.director as d, SXDS.h_f_g as fg where d.objectId=fg.objectId ORDER BY d.chunkId,d.subChunkId"
results = service.search(query)

results

<Table length=10>
chunkId         dec               objectId      ... subChunkId2 tract
 int32        float64              int64        ...    int32    int32
------- ------------------- ------------------- ... ----------- -----
  30273  -5.671077252404507 3036354136661559483 ...           1  8523
  30273 -5.6688598363965585 3036354136661559485 ...           1  8523
  30273  -5.676487568140274 3036354136661559507 ...           1  8523
  30273 -5.6736059186377075 3036354136661559528 ...           1  8523
  30273  -5.674675879397784 3036354136661559523 ...           2  8523
  30273   -5.67456978976135 3036345340568535150 ...           2  8523
  30273  -5.674907490049822 3036345340568535143 ...           2  8523
  30273  -5.675130456865955 3036345340568535131 ...           2  8523
  30273  -5.674921997791494 3036345340568535137 ...           2  8523
  30273 -5.6748531529010275 3036345340568535141 ...           2  8523

In [7]:
print(type(results))
table=results.to_table()
print(type(table))

In [8]:
query="select objectId from SXDS.director order by objectId limit 1000000"
results = service.search(query)
table = results.to_table()
print(table)

In [ ]:
objectIds=table['objectId'].data
print(objectIds[0],max(objectIds),min(objectIds))

In [12]:
import numpy
chunks = numpy.array_split(numpy.array(objectIds),100)
import datetime
i=0
from astropy.table import QTable, Table, Column, vstack
fullResults=Table()
for chunk in chunks:
    print(i,len(chunk),datetime.datetime.now())
    
    inClause=','.join(str(objId) for objId in chunk) #numpy.array2string(chunk, separator=",")
    #print(inClause)
    query="select * from SXDS.director where objectId in ("+inClause+")"
    #print(query)
    results = service.search(query)
    table = results.to_table()
    if i==0:
        fullResults=table
    else:
        fullResults=(vstack([fullResults, table]))
    #print(table)
    i+=1
    if i==300:
        break
print(len(fullResults))

0 10000 2022-12-01 09:06:49.799289
1 10000 2022-12-01 09:07:05.189538
2 10000 2022-12-01 09:07:21.382488
3 10000 2022-12-01 09:07:36.705267
4 10000 2022-12-01 09:07:51.459631
5 10000 2022-12-01 09:08:07.153964
6 10000 2022-12-01 09:08:24.148682
7 10000 2022-12-01 09:08:41.014212
8 10000 2022-12-01 09:08:57.493699
9 10000 2022-12-01 09:09:14.148971
10 10000 2022-12-01 09:09:28.360350
11 10000 2022-12-01 09:09:45.053914
12 10000 2022-12-01 09:10:00.781937
13 10000 2022-12-01 09:10:16.380475
14 10000 2022-12-01 09:10:31.863677
15 10000 2022-12-01 09:10:46.699005
16 10000 2022-12-01 09:11:01.706004
17 10000 2022-12-01 09:11:17.215775
18 10000 2022-12-01 09:11:32.325222
19 10000 2022-12-01 09:11:48.594884
20 10000 2022-12-01 09:12:03.485288
21 10000 2022-12-01 09:12:19.141816
22 10000 2022-12-01 09:12:34.278045
23 10000 2022-12-01 09:12:50.163523
24 10000 2022-12-01 09:13:05.530389
25 10000 2022-12-01 09:13:21.153146
26 10000 2022-12-01 09:13:36.044783
27 10000 2022-12-01 09:13:52.615549
28

In [ ]:
import pyvo
tap = pyvo.dal.TAPService('https://rsp.lsst.ac.uk/api/tap')
tap.run_sync('select count(*) from SXDS.director')